In [1]:
import pandas as pd

In [ ]:
ah = pd.read_excel('ds/Asentamiento Histo.xlsx')
lin = pd.read_excel('ds/LIN.xlsx')

In [ ]:
ah = ah.drop(['Unnamed: 5', 'clave_autodenom', 'Unnamed: 7',
       'nom_autodenom_2008', 'nom_autodenom_1actualizacion',
       'Trans_fonetica_2008', 'Trans_fonetica_1actualización',
       'clave_ent_2000', 'clave_ent_2010', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'loc2015', 'tieneTocayas2000',
       'bandera2000', 'mayorHLI2000', 'HLI_2000', 'HLI_2005', 'HLI_2010',
       'edomunloc2000', 'edomunloc2010', 'clave_unica2000', 'clave_unica2010',
       'clave_unica2015', 'NDRIESGO', 'cambios_DI',
       'Claves para marcas de densidad ', 'Notas', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44'],axis=1)

In [ ]:
ah = ah.dropna()

In [ ]:
lin.head()

In [ ]:
lin.columns

In [ ]:
ah.head()

In [ ]:
temporal = lin[['AH/M', 'Población de 3 años y más que habla lengua indígena', 'Hombres', 'Mujeres', 'AH/M.1']]

In [ ]:
temporal.shape

In [ ]:
print(lin.columns)
print(ah.columns)

In [ ]:
#df2.set_index('player').subtract(df1.set_index('player'))

diff_df = pd.merge(lin[['Lengua indígena', 'Entidad federativa', 'Municipio', 'Localidad']], ah, how='outer', indicator='Exist')

In [ ]:
diff_df.head()

In [ ]:
diff_df = pd.concat([diff_df,temporal], axis=1)

In [ ]:
diff_df.to_excel("out/resultado.xlsx")